In [11]:
# Inital state probability matrix, given
PI = [0.5,0.5]
# PI = {'Healthy': 0.5, 'Depressed': 0.5}
# state transition probability matrix, given
A = [[0.8,0.2],[0.001,0.999]]
# A = {
#     'Healthy' : {'Healthy': 0.8, 'Depressed': 0.2},
#     'Depressed' : {'Healthy': 0.001, 'Depressed': 0.999},
# }
# Emission or observation probability matrix, given
B = [[0.5,0.1,0.3,0.1,0.0],[0.05,0.35,0.2,0.2,0.2]]
# B = {
#     'Healthy' : {'Movement': 0.5, 'Passive': 0.1, 'Active' : 0.3, 'Texting' : 0.1, 'Access' : 0.0},
#     'Depressed' : {'Movement': 0.05, 'Passive': 0.35, 'Active' : 0.2, 'Texting' : 0.2, 'Access' : 0.2},
# }

# state codes of the observed states, dummy
# Observables = [0,1,2,3,4]
# Observables = ["Movement","Passive","Active","Texting","Access"]
#Observables = ["Movement", "Texting", "Movement", "Texting"]
#states = ["Healthy","Depressed"]
states = [0, 1]
Observables = [0,3,0,3]
#Observables = {"Movement":0, "Passive Social":1, "Active Social":2, "Texting":3, "Access Psych Site":4}

In [12]:
rows = [0] * 3

In [13]:
rows

[0, 0, 0]

In [14]:
matrix = [rows] * 3

In [15]:
matrix

[[0, 0, 0], [0, 0, 0], [0, 0, 0]]

# Filtering

In [113]:
import numpy as np 
from numpy import matrix

def forward(PI, A, B, Observables):
    #Note: alpha_1(i) = pi_i(b_i)(O_1)
    #i.e. alpha_1(i) = PI[i](B[i])(Observables[1])
    #dim of alpha_matrix is n_A x n_Observables
    rows = [0] * len(Observables)
    alpha_matrix = [rows] * len(A)
    
    for t,value in enumerate(Observables):
        for state in states: 
            if t == 0:
                alpha_matrix[state][t] = PI[t]*B[state][t]* B[state][value]
            else: 
                alpha_matrix[state][t] = sum(alpha_matrix[l][t-1] * A[state][l] for l in states)  * B[state][value]
            
    print(alpha_matrix)
    alpha_matrix = np.array(alpha_matrix).transpose()


    return alpha_matrix #(normalized forward probability matrix), c (one-dimensional normalization (scaling) coefficient array)

In [114]:
forward(PI, A, B, Observables)

[[0.0012500000000000002, 0.00025000000000000006, 1.2500000000000006e-05, 2.500000000000001e-06], [0.0012500000000000002, 0.00025000000000000006, 1.2500000000000006e-05, 2.500000000000001e-06]]


array([[1.25e-03, 1.25e-03],
       [2.50e-04, 2.50e-04],
       [1.25e-05, 1.25e-05],
       [2.50e-06, 2.50e-06]])

In [126]:
def forward_normalized(PI, A, B, Observables):
    alpha = forward(PI, A, B, Observables)
    c = [0] * len(Observables)
    for i,i_value in enumerate(alpha):
        for j,j_value in enumerate(i_value):
            c[j] += i_value[j]
    
    for i in range(len(alpha)):
        for j in range(len(alpha[i])):
            alpha[i][j] = alpha[i][j] * c[j]
    
    return alpha, c

In [127]:
forward_normalized(PI, A, B, Observables)

[[0.0012500000000000002, 0.00025000000000000006, 1.2500000000000006e-05, 2.500000000000001e-06], [0.0012500000000000002, 0.00025000000000000006, 1.2500000000000006e-05, 2.500000000000001e-06]]


(array([[1.89375e-06, 1.89375e-06],
        [3.78750e-07, 3.78750e-07],
        [1.89375e-08, 1.89375e-08],
        [3.78750e-09, 3.78750e-09]]), [0.001515, 0.001515, 0, 0])

# Evaluation

In [14]:
def evaluation_unnormalized(alpha):
    return #probability

In [ ]:
def evaluation_normalized(c):
    return probability

# Backward Probability

In [115]:
def backward(PI, A, B, Observables):
    
    beta = []
    b_prev = {}
    for i, value in enumerate(reversed(Observables)):
        b_curr = {}
        for state in states:
            if i == 0:
                b_curr[state] = 1 
            else:
                b_curr[state] = sum(A[state][l] * B[l][value] * b_prev[l] for l in states)

        beta.insert(0,list(b_curr.values()))
        b_prev = b_curr

    #p_beta = sum(PI[l] * B[l][Observables[0]] * b_curr[l] for l in states)
    
    return beta #no of initial states * length of observation sequence

In [116]:
backward(PI, A, B, Observables)

[[0.014028409100000003, 0.0005229484545000001],
 [0.03481800000000001, 0.01012091],
 [0.41000000000000003, 0.05045],
 [1, 1]]

In [128]:
def backward_normalized(PI, A, B, Observables):
    alpha,c = forward_normalized(PI, A, B, Observables)
    beta = backward(PI, A, B, Observables)


    for i in range(len(beta)):
        for j in range(len(beta[i])):
            beta[i][j] = beta[i][j] * c[j]
            
    return beta

In [129]:
backward_normalized(PI, A, B, Observables)

[[0.0012500000000000002, 0.00025000000000000006, 1.2500000000000006e-05, 2.500000000000001e-06], [0.0012500000000000002, 0.00025000000000000006, 1.2500000000000006e-05, 2.500000000000001e-06]]


[[2.1253039786500007e-05, 7.922669085675002e-07],
 [5.274927000000002e-05, 1.5333178650000003e-05],
 [0.0006211500000000001, 7.643175000000001e-05],
 [0.001515, 0.001515]]

# Smoothing by Forward-Backward Algorithm

In [ ]:
def smoothed_probability(alpha, beta, noOfStates, noOfTimeSteps):
    return matrix